# Import libraries

In [1]:
import numpy as np
import cv2
import os
import pandas as pd

# Data Acquistion

In [22]:
# folder_path = './Traffic Sign/DATA'
# dict = {}
# for filename in os.listdir(folder_path): # Reading folder names from Cars folder
#     image_list = []
#     curr_path = folder_path+'/'+filename
#     for imagename in os.listdir(curr_path): # Reading image names from each car type
#         image_list.append(cv2.imread(curr_path+'/'+imagename))
#     dict[filename] = image_list
# dict

{'0': [array([[[236, 232, 227],
          [236, 232, 227],
          [237, 233, 228],
          ...,
          [ 64,  90,  80],
          [ 70,  98,  88],
          [ 70, 100,  88]],
  
         [[236, 232, 227],
          [236, 232, 227],
          [236, 232, 227],
          ...,
          [ 79, 103,  94],
          [ 73, 101,  90],
          [ 71, 101,  89]],
  
         [[236, 231, 227],
          [236, 231, 227],
          [236, 231, 228],
          ...,
          [ 76, 101,  89],
          [ 60,  87,  75],
          [ 64,  92,  81]],
  
         ...,
  
         [[102, 110,  94],
          [103, 112,  96],
          [151, 158, 142],
          ...,
          [ 72, 102,  91],
          [ 75, 105,  94],
          [ 66,  96,  85]],
  
         [[102, 113,  97],
          [ 98, 109,  93],
          [102, 112,  96],
          ...,
          [ 75, 105,  94],
          [ 72, 101,  90],
          [ 68,  97,  86]],
  
         [[122, 132, 116],
          [115, 125, 109],
          [112, 119

In [2]:
import os
import cv2
import numpy as np

# Define the fixed size for images
fixed_size = (100, 100)  # Change this to your desired size
max_images_per_class = 100  # Change this to the maximum number of images per class

# Define the folder path
folder_path = './Traffic Sign/DATA'

# Initialize a dictionary to store images
data_dict = {}

# Function to perform image augmentation
def augment_images(image):
    augmented_images = []
    
    # Perform rotation
    for angle in range(-15, 16, 5):
        rotated = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
        augmented_images.append(cv2.resize(rotated, fixed_size))
        
    # Perform flipping
    flipped_lr = cv2.flip(image, 1)
    flipped_ud = cv2.flip(image, 0)
    augmented_images.extend([cv2.resize(flipped_lr, fixed_size), cv2.resize(flipped_ud, fixed_size)])
    
    # Perform scaling
    for scale in np.linspace(0.9, 1.1, 5):
        scaled = cv2.resize(image, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
        augmented_images.append(cv2.resize(scaled, fixed_size))
        
    return augmented_images

# Pad or crop images to ensure uniform length
def pad_or_crop_images(images):
    max_length = max(len(img) for img in images)
    padded_images = []
    for img in images:
        diff = max_length - len(img)
        if diff > 0:
            # Pad the image
            padding = [(diff // 2, diff // 2 + diff % 2), (0, 0), (0, 0)]
            padded_img = np.pad(img, padding, mode='constant')
        else:
            # Crop the image
            cropped_img = img[:max_length]
        padded_images.append(padded_img if diff > 0 else cropped_img)
    return padded_images

# Iterate through each folder (class)
for filename in os.listdir(folder_path):
    image_list = []
    curr_path = os.path.join(folder_path, filename)
    count = 0  # Counter to limit the number of images per class
    
    # Read images from each class folder
    for imagename in os.listdir(curr_path):
        if count >= max_images_per_class:
            break
        image = cv2.imread(os.path.join(curr_path, imagename))
        
        # Augment images
        augmented_images = augment_images(image)
        image_list.extend(augmented_images)
        
        count += 1  # Increment the counter
        
    # Pad or crop images to ensure uniform length
    image_list = pad_or_crop_images(image_list)
        
    # Store the augmented images for each class
    data_dict[filename] = image_list

# Now data_dict contains augmented images for each class, all with uniform length



In [3]:
for key in data_dict:
    print(len(data_dict[key]))
    

1400
1400
1400
1400
1400
1400
1400


In [4]:
# Preprocessing function to apply normalization, histogram equalization, and denoising
def preprocess_image(image):
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Normalization
    normalized_image = cv2.normalize(gray_image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    
    # Histogram Equalization
    equalized_image = cv2.equalizeHist(gray_image)
    
    # Denoising
    denoised_image = cv2.fastNlMeansDenoising(equalized_image, None, h=10, searchWindowSize=21)
    
    return denoised_image


In [5]:
import os
import cv2
import numpy as np

# Define the fixed size for images
fixed_size = (100, 100)  # Change this to your desired size
max_images_per_class = 100  # Change this to the maximum number of images per class

# Define the folder path
folder_path = './Traffic Sign/DATA'

# Initialize a dictionary to store preprocessed images
preprocessed_data_dict = {}

# Function to perform image augmentation
def augment_images(image):
    augmented_images = []
    
    # Perform rotation
    for angle in range(-15, 16, 5):
        rotated = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
        augmented_images.append(cv2.resize(rotated, fixed_size))
        
    # Perform flipping
    flipped_lr = cv2.flip(image, 1)
    flipped_ud = cv2.flip(image, 0)
    augmented_images.extend([cv2.resize(flipped_lr, fixed_size), cv2.resize(flipped_ud, fixed_size)])
    
    # Perform scaling
    for scale in np.linspace(0.9, 1.1, 5):
        scaled = cv2.resize(image, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
        augmented_images.append(cv2.resize(scaled, fixed_size))
        
    return augmented_images

# Pad or crop images to ensure uniform length
def pad_or_crop_images(images):
    max_length = max(len(img) for img in images)
    padded_images = []
    for img in images:
        diff = max_length - len(img)
        if diff > 0:
            # Pad the image
            padding = [(diff // 2, diff // 2 + diff % 2), (0, 0), (0, 0)]
            padded_img = np.pad(img, padding, mode='constant')
        else:
            # Crop the image
            cropped_img = img[:max_length]
        padded_images.append(padded_img if diff > 0 else cropped_img)
    return padded_images

# Preprocessing function to apply normalization, histogram equalization, and denoising
# def preprocess_image(image):
#     # Normalization
#     normalized_image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    
#     # Histogram Equalization (assuming grayscale)
#     gray_image = cv2.cvtColor(normalized_image, cv2.COLOR_BGR2GRAY)
#     equalized_image = cv2.equalizeHist(gray_image)
    
#     # Denoising
#     denoised_image = cv2.fastNlMeansDenoisingColored(equalized_image, None, 10, 10, 7, 21)
    
#     return denoised_image
# Preprocessing function to apply normalization, histogram equalization, and denoising
# def preprocess_image(image):
#     # Convert to grayscale
#     gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
#     # Normalization
#     normalized_image = cv2.normalize(gray_image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    
#     # Histogram Equalization
#     equalized_image = cv2.equalizeHist(normalized_image)
    
#     # Denoising
#     denoised_image = cv2.fastNlMeansDenoising(equalized_image, None, h=10, searchWindowSize=21)
    
#     return denoised_image



# Preprocessing function to apply normalization, histogram equalization, and denoising
def preprocess_image(image):
    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Normalization
    normalized_image = cv2.normalize(gray_image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    
    # Histogram Equalization
    equalized_image = cv2.equalizeHist(gray_image)
    
    # Denoising
    denoised_image = cv2.fastNlMeansDenoising(equalized_image, None, h=10, searchWindowSize=21)
    
    return denoised_image

# Iterate through each folder (class)
for filename in os.listdir(folder_path):
    image_list = []
    curr_path = os.path.join(folder_path, filename)
    count = 0  # Counter to limit the number of images per class
    
    # Read images from each class folder
    for imagename in os.listdir(curr_path):
        if count >= max_images_per_class:
            break
        image = cv2.imread(os.path.join(curr_path, imagename))
        
        # Preprocess image
        preprocessed_image = preprocess_image(image)
        
        # Augment images
        augmented_images = augment_images(preprocessed_image)
        image_list.extend(augmented_images)
        
        count += 1  # Increment the counter
        
    # Pad or crop images to ensure uniform length
    image_list = pad_or_crop_images(image_list)
        
    # Store the augmented and preprocessed images for each class
    preprocessed_data_dict[filename] = image_list

# Now preprocessed_data_dict contains augmented and preprocessed images for each class


In [ ]:
# import cv2

# # Function to compute HOG features
# def compute_hog_features(image):
#     if image is None:
#         print("Error: Input image is empty.")
#         return None
    
#     # Convert image to grayscale
#     gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
#     # Compute HOG features
#     hog = cv2.HOGDescriptor()
#     hog_features = hog.compute(gray_image)
    
#     return hog_features

# # Function to compute LBP features
# def compute_lbp_features(image):
#     if image is None:
#         print("Error: Input image is empty.")
#         return None
    
#     # Convert image to grayscale
#     gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
#     # Compute LBP features
#     lbp = cv2.LBP_create()
#     lbp_features = lbp.compute(gray_image)
    
#     return lbp_features

# # Load an example image
# image = cv2.imread('example_image.jpg')

# # Compute HOG features
# hog_features = compute_hog_features(image)

# # Compute LBP features
# lbp_features = compute_lbp_features(image)


Error: Input image is empty.
Error: Input image is empty.


Sure, let's break down each method:

### Histogram of Oriented Gradients (HOG):

HOG is a feature descriptor that captures the distribution of gradient orientations in an image. It's widely used in object detection and classification tasks. Here's how you can implement it using OpenCV and NumPy:

1. **Convert Image to Grayscale**: HOG works on grayscale images. So, the first step is to convert the input color image to grayscale.
   
2. **Compute Gradients**: Compute the gradients (both magnitude and orientation) of the grayscale image using techniques like Sobel operator.
   
3. **Divide Image into Cells**: Divide the image into small cells, typically 8x8 pixels.
   
4. **Compute Histograms**: For each cell, compute a histogram of gradient orientations. These histograms capture the distribution of gradient orientations within each cell.
   
5. **Block Normalization**: Normalize the histograms within each block (a group of cells). This normalization helps in achieving invariance to changes in illumination and contrast.
   
6. **Concatenate Block Features**: Concatenate the normalized block features to form the final feature vector for the image.

OpenCV provides a built-in function `cv2.HOGDescriptor()` to compute HOG features.

### Local Binary Patterns (LBP):

LBP is another feature descriptor used for texture classification. It encodes the local structure of an image by comparing each pixel with its neighbors. Here's how you can implement it using OpenCV and NumPy:

1. **Convert Image to Grayscale**: Similar to HOG, LBP also operates on grayscale images.
   
2. **Compute LBP for Each Pixel**: For each pixel in the image, compare its intensity value with the intensity values of its neighbors. Encode the result as a binary number.
   
3. **Histogram of LBP**: Compute a histogram of the LBP values over the entire image or its regions. This histogram represents the distribution of local patterns in the image.
   
4. **Concatenate Histograms**: Concatenate the histograms from different regions (if applicable) to form the final feature vector for the image.

OpenCV provides a function `cv2.calcHist()` to compute histograms, which can be used to compute the LBP histogram.

### Utilizing Pre-trained CNNs for Feature Extraction:

Deep CNNs, especially those pretrained on large datasets like ImageNet, learn rich hierarchical features that are useful for various computer vision tasks. Here's how you can extract features from pre-trained CNNs using TensorFlow's Keras API:

1. **Load Pre-trained Model**: Load a pre-trained CNN model such as VGG, ResNet, or Inception from TensorFlow's Keras API. These models are trained on large datasets and capture rich image representations.
   
2. **Remove Classification Head**: Remove the fully connected layers (classification head) from the pre-trained model. We're interested in the features learned by the convolutional layers.
   
3. **Feature Extraction**: Pass the input images through the modified pre-trained model and extract features from one of the intermediate layers. These features serve as the representation of the input images.
   
4. **Use Extracted Features**: The extracted features can be used directly for classification tasks or as input to other machine learning models.

TensorFlow's Keras API provides an easy way to load pre-trained models and extract features using the `predict()` method.

### Comparison and Reasons for Classification Task:

- **HOG and LBP**: These methods are suitable for tasks where local texture and shape information are important, such as object detection and texture classification. They are computationally efficient and provide interpretable features. However, they may not capture high-level semantic information present in deep CNN features.

- **Pre-trained CNNs**: Pre-trained CNN features are suitable for tasks where high-level semantic information is crucial, such as image classification, object recognition, and scene understanding. They capture rich, hierarchical representations learned from large-scale datasets. However, they may require more computational resources for both training and inference.

The choice between these methods depends on the specific requirements of the classification task, including the nature of the input data, computational constraints, and desired level of interpretability.

In [55]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)

# Define the layers from which to extract features
feature_extractor = tf.keras.Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)

# Function to extract features from images using pre-trained CNN
def extract_cnn_features(images):
    # Preprocess images
    preprocessed_images = []
    for image in images:
        preprocessed_image = preprocess_input(image)  # Preprocess input images
        preprocessed_images.append(preprocessed_image)
    preprocessed_images = np.array(preprocessed_images)
    
    # Extract features
    features = feature_extractor.predict(preprocessed_images)
    
    return features

# Example usage:
# Assuming img_dict is a 2D dictionary containing RGB images in the array

# Extract features from images
extracted_features = {}
for category, images in dict.items():
    features = extract_cnn_features(images)
    extracted_features[category] = features


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (314,) + inhomogeneous part.